https://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides1.pdf

In [ ]:
from osgeo import ogr
import sys

In [2]:
fileName = 'Cobertura_uso_terra_Brasil-dissolve.shp'

In [3]:
filePath = '/Users/francescoghizzo/Documents/DEEP ESG/Bases de Dados/IBGE/Cobertura_uso_terra_Brasil_serie_revisada/'

In [4]:
file = filePath + fileName

<h2>Opening a layer (shapefile)</h2>

A driver is an object that knows how to interact with a certain data type (such as a shapefile).

You need an appropriate driver in order to read or write data (need it explicitly for write).

To get the desired driver:

In [5]:
driver = ogr.GetDriverByName('ESRI Shapefile')

Grab the driver for read operations so it is available for writing.

The Driver <b>Open()</b> method returns a DataSource object

<b>Open(&lt;filename>, &lt;update>;)</b>
    
where <update> is 0 for read-only, 1 for writeable

In [ ]:
dataSource = driver.Open(file, 0)
if dataSource is None:
    print('Could not open' + fileName)
    sys.exit(1)

To avoid specifying entire path for filenames, set working directory with:

<b>os.chdir(&lt;directory_path>)</b>

In [7]:
import os

In [8]:
os.chdir(filePath)

In [9]:
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(fileName, 0)
if dataSource is None:
    print('Could not open' + fileName)
    sys.exit(1)

Use <b>GetLayer(&lt;index>)</b> on a DataSource to get a Layer object:

In [10]:
layer = dataSource.GetLayer()

&lt;index> is always 0 and optional for shapefiles:

In [11]:
layer = dataSource.GetLayer(0)

<h2>Getting info about the layer</h2>

Get the name of the layer:

In [30]:
name = layer.GetName()
print('Layer name:', name)

Layer name: Cobertura_uso_terra_Brasil-dissolve


In [31]:
geomType = layer.GetGeomType()  # GeomType = point, linestring, polygon o GeomType = geometry, geography, topology?

Get the number of features in the layer:

In [14]:
numFeatures = layer.GetFeatureCount()
print('Feature count:', numFeatures)

Feature count: 12


In [15]:
layer.ResetReading()

count = 0
feature = layer.GetNextFeature()
while feature is not None:
    count += 1
    feature = layer.GetNextFeature()
print('Feature count:', count)

Feature count: 12


Get the extent as a tuple:

In [16]:
extent = layer.GetExtent()

In [17]:
print('Extent:', extent)

Extent: (-74.00005989467657, -28.710965592652176, -33.878091759421565, 5.273917089392972)


In [18]:
print('UL:', extent[0], extent[3])

UL: -74.00005989467657 5.273917089392972


In [19]:
print('LR:', extent[1], extent[2])

LR: -28.710965592652176 -33.878091759421565


If we know the FID (offset) of a feature, we can use <b>GetFeature(&#60;index&#62;)</b> on the Layer:

In [27]:
feature = layer.GetFeature(0)

Or we can loop through all of the features:

In [21]:
feature = layer.GetNextFeature()
while feature:
    # do something here
    feature = layer.GetNextFeature()
layer.ResetReading()    #need if looping again

<h2>Getting a feature's attributes</h2>

Get a list of the feature's attribute fields:

In [25]:
layerDefinition = layer.GetLayerDefn()
layerColNames = [layerDefinition.GetFieldDefn(i).GetName() for i in range(layerDefinition.GetFieldCount())]
print('Column names:', layerColNames)

Column names: ['USO2020', 'NOME', 'id']


Feature objects have a <b>GetField(&#60;name&#62;)</b> method which returns the value of that attribute field.

There are variations, such as <b>GetFieldAsString(&#60;name&#62;)</b> and
    <b>GetFieldAsInteger(&#60;name&#62;)</b>:

In [28]:
id = feature.GetField('id')
id = feature.GetFieldAsString('id')

<h2>Getting a feature's geometry</h2>

Feature objects have a method called <b>GetGeometryRef()</b> which returns a
Geometry object (could be Point, Polygon, etc)

In [29]:
geometry = feature.GetGeometryRef()

Point objects have <b>GetX()</b> and <b>GetY()</b> methods

In [ ]:
x = geometry.GetX()
y = geometry.GetY()

<h2>Destroying objects</h2>

For memory management purposes we need to make sure that we get rid of things such as features when done with them:

In [172]:
feature.Destroy()

Also need to close DataSource objects when done with them

In [173]:
dataSource.Destroy()

<h2>Adding fields</h2>

We cannot add fields to non-empty shapefiles. 
Shapefiles need at least one attribute field.

We need a `FieldDefn` object first.
You can copy one from an existing feature with <b>GetFieldDefnRef(&#60;field_index&#62;)</b> or <b>GetFieldDefnRef(&#60;field_name&#62;)</b>:

In [33]:
fieldDefn = feature.GetFieldDefnRef(0)
fieldDefn = feature.GetFieldDefnRef('id')

Now create a field on the layer using the `FieldDefn` object and <b>CreateField(&#60;fieldDefn&#62;)</b>:

In [35]:
layer.CreateField(fieldDefn)

ERROR 6: CreateField : unsupported operation on a read-only datasource.


6